In [1]:
 %pip install -q langchain langchain_experimental genai python-dotenv langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.9/831.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of th

In [5]:
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load environment variables
from dotenv import load_dotenv
import os

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

# Set a random seed for reproducibility
np.random.seed(360)

Generate Sample Data
In this section, we create a sample dataset of products. This includes generating dates,  makes, products, colors, and other relevant information.

In [20]:
#Generate sample data
n_rows = 1000

# Generate dates
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# Define data categories
makes = ['Nike', 'Samsung', 'Apple', 'Sony', 'LG', 'Adidas', 'Dell', 'Panasonic', 'Puma', 'HP']
Product = ['Running Shoes', 'Smart TV', 'iPhone 15', 'Bluetooth Speaker', 'Sweatshirt', 'Laptop', 'Headphones', 'Tablet', 'Camera', 'Printer']
colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# Create the dataset
data = {
    'Date': dates,
    'Make': np.random.choice(makes, n_rows),
    'Model': np.random.choice(Product, n_rows),
    'Color': np.random.choice(colors, n_rows),
    'Year': np.random.randint(2013, 2025, n_rows),
    'price': np.random.uniform(200, 8000, n_rows).round(2),
    'Quantity': np.random.randint(1, 10, n_rows),
    'SalesPerson': np.random.choice(['Alice', 'Bob', 'King', 'David', 'Tobi'], n_rows)
}

# Create DataFrame and sort by date
df = pd.DataFrame(data).sort_values('Date')

# Add a 'Category' column
# Categories mapped to the Product list
product_categories = {
    'Running Shoes': 'Apparel',
    'Smart TV': 'Electronics',
    'iPhone 15': 'Electronics',
    'Bluetooth Speaker': 'Electronics',
    'Sweatshirt': 'Apparel',
    'Laptop': 'Electronics',
    'Headphones': 'Electronics',
    'Tablet': 'Electronics',
    'Camera': 'Electronics',
    'Printer': 'Electronics'
}

df['Category'] = df['Model'].map(product_categories)

# Add 'Payment Method' column
payment_methods = ['Credit Card', 'Cash', 'Online Payment', 'Debit Card']
df['Payment Method'] = np.random.choice(payment_methods, n_rows)

# Add 'Region' column
regions = ['North', 'South', 'East', 'West', 'Central']
df['Region'] = np.random.choice(regions, n_rows)


# Display sample data and statistics
print("\nFirst few rows of the generated data:")
print(df.head())

print("\nDataFrame info:")
df.info()

print("\nSummary statistics:")
print(df.describe())


First few rows of the generated data:
        Date   Make              Model   Color  Year    price  Quantity  \
0 2022-01-01     LG             Tablet  Silver  2014  2534.30         8   
1 2022-01-02   Dell          iPhone 15    Blue  2024  5583.96         4   
2 2022-01-03  Apple         Headphones   White  2017  6152.74         8   
3 2022-01-04  Apple  Bluetooth Speaker   White  2023  3213.19         6   
4 2022-01-05     HP            Printer    Blue  2016  4696.43         3   

  SalesPerson     Category Payment Method   Region  
0         Bob  Electronics     Debit Card     West  
1        King  Electronics     Debit Card  Central  
2        Tobi  Electronics     Debit Card    South  
3       David  Electronics     Debit Card  Central  
4         Bob  Electronics           Cash     West  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------      

Create Data Analysis Agent
Here, we create a Pandas DataFrame agent using LangChain. This agent will be capable of analyzing our dataset and answering questions about it.

In [21]:
# Create the Pandas DataFrame agent
agent = create_pandas_dataframe_agent(
    ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0),
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
print("Data Analysis Agent is ready. You can now ask questions about the data.")

Data Analysis Agent is ready. You can now ask questions about the data.


Define Question-Asking Function
This function allows us to easily ask questions to our data analysis agent and display the results.

In [22]:
def ask_agent(question):
    """Function to ask questions to the agent and display the response"""
    response = agent.run({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("---")

Example Questions
Here are some example questions you can ask the data analysis agent. You can modify these or add your own questions to analyze the dataset.

In [24]:
# Example questions
ask_agent("What are the column names in this dataset?")

ask_agent("How many rows are in this dataset?")

ask_agent("What is the average price of products sold?")

ask_agent("What is the total quantity of products sold?")

ask_agent("what is the average price of Apple products sold in South regions")

ask_agent("What is the average price of Apple products sold?")



> Entering new AgentExecutor chain...
Thought: I need to extract the column names from the dataframe.
Action: python_repl_ast
Action Input: print(df.columns)Index(['Date', 'Make', 'Model', 'Color', 'Year', 'price', 'Quantity',
       'SalesPerson', 'Category', 'Payment Method', 'Region'],
      dtype='object')
I have successfully extracted the column names.
Final Answer: Date, Make, Model, Color, Year, price, Quantity, SalesPerson, Category, Payment Method, Region


> Finished chain.
Question: What are the column names in this dataset?
Answer: Date, Make, Model, Color, Year, price, Quantity, SalesPerson, Category, Payment Method, Region
---


> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I can use the `len()` function or the `df.shape` attribute. I'll use `df.shape` to get the number of rows and columns.
Action: python_repl_ast
Action Input: `df.shape`(1000, 11)The dataframe `df` has 1000 rows and 11 columns. Therefore, the number of rows